<a href="https://colab.research.google.com/github/galrat/parsing/blob/main/parsing_from_hh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code gets data from hh.ru.

input data is url with search results, for example, "Аналитик" in title of vacancy, remote type of work etc.

output data is csv-file with description of all parsed vacencies

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# defs

In [2]:
def get_vacancy_urls(soup):
  vacancy_urls = [a['href'] for a in soup.find_all('a', class_='serp-item__title', href=True)]
  return vacancy_urls

def get_vacancy_info(soup, vac_ulr):
  vacancy_data_line = []

  title = soup.find('div', class_='vacancy-title').find('h1').text

  try:
    salary = soup.find('div', class_='bloko-v-spacing bloko-v-spacing_base-4').find_next_sibling().text.replace('\xa0', '')
  except:
    salary = 'не указана'

  try:
    details = soup.find('div', class_='bloko-columns-row').find_all('p', class_="vacancy-description-list-item")
    details = ('; \n'.join([i.text.strip() for i in details]))
  except:
    details = ''


  vacancy_details = soup.find('div', class_='g-user-content')
  try:
    job_responsibilities = vacancy_details.text.split('Обязанности:')[1].strip()
  except:
    job_responsibilities = ''
  #print('job_responsibilities', job_responsibilities)

  skill_data = soup.find_all('div', class_='vacancy-section')
  try:
    skills = ''
    for skill in skill_data:
      if 'Ключевые навыки' in skill.text:
        for sk in skill.find_all('div', class_='bloko-tag bloko-tag_inline'):
          skills += sk.text + '; \n'
  except:
    skills = ''

  try:
    add_data = soup.find('div', class_='tmpl_hh_wrapper').text
  except:
    add_data = ''

  vacancy_data_line.append([title, salary, details, vacancy_details.text, job_responsibilities, skills, add_data, vac_ulr])

  return vacancy_data_line[0]
#print(data.split('Ключевые навыки')[1])

# main code

1. Беру адрес
2. запускаю подпрограмму, которая по адресу находит все ссылки на вакансии
3. запускаю подпрограмму, которая по ссылкам на вакансии извлекает всю релевантную инфу
4. добавляю &page=1 к адресу, пытаюсь выполнить этап 2, если сработал триггер на завершение, то break


In [3]:
# адрес страницы с результатами поиска
page_url = 'https://hh.kz/search/vacancy?L_save_area=true&search_field=name&schedule=remote&items_on_page=50&experience=noExperience&text=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&enable_snippets=false'


In [4]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
columns = ['title', 'salary', 'details', 'vacancy_details', 'job_responsibilities', 'skills', 'add_data', 'vac_ulr']
vacancy_data = []
counter = 0
new_page_url = page_url
while True:
  print(new_page_url, '\n counter', counter)
  content = requests.get(new_page_url, headers=headers)
  if (content.status_code) == 404:
    print('parsing is finished')
    break
  if counter == 10:
    print('parsing is finished')
    break

  # get first part of urls from first hh page
  page_soup = BeautifulSoup(content.text, 'lxml')
  page_urls = get_vacancy_urls(page_soup)
  print('page_urls', page_urls)
  print('==================================\n')

  if page_urls == []:
    print('parsing is finished')
    break

  # get data from each url
  for vacancy_url in page_urls[:]:
    try:
      print(vacancy_url)
      vac_content = requests.get(vacancy_url, headers=headers)
      vacancy_soup = BeautifulSoup(vac_content.text, 'lxml')
      vacancy_data.append(get_vacancy_info(vacancy_soup, vacancy_url))
    except:
      vacancy_data.append(['', '', '', '', '', '', '', vacancy_url])
  vacancy_data_df = pd.DataFrame(vacancy_data, columns=columns)
  vacancy_data_df.to_csv('/content/drive/MyDrive/parsed_hh_3.csv')

  counter += 1
  new_page_url = page_url + '&page=' + str(counter)


https://hh.kz/search/vacancy?L_save_area=true&search_field=name&schedule=remote&items_on_page=50&experience=noExperience&text=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&enable_snippets=false 
 counter 0
page_urls ['https://hh.kz/vacancy/87179946?from=vacancy_search_list&query=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&hhtmFrom=vacancy_search_list', 'https://hh.kz/vacancy/88171159?from=vacancy_search_list&query=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&hhtmFrom=vacancy_search_list', 'https://hh.kz/vacancy/90677195?from=vacancy_search_list&query=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&hhtmFrom=vacancy_search_list', 'https://hh.kz/vacancy/90797243?from=vacancy_search_list&query=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&hhtmFrom=vacancy_search_list', 'https://hh.kz/vacancy/90797272?from=vacancy_search_list&query=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&hhtmFrom=vacancy_search_list', 'https://hh.kz/vacancy/91239733?from=vacancy_search_list&query=%D

In [5]:
vacancy_data_df

,title,salary,details,vacancy_details,job_responsibilities,skills,add_data,vac_ulr
0,Аналитик,от 30000 до 150000 ₽ до вычета налогов,Требуемый опыт работы: не требуется; \nПолная ...,✨Присоединяйтесь к Архитекторам Будущего Цифро...,,Бизнес-анализ; \nАналитическое мышление; \nСта...,,https://hh.kz/vacancy/87179946?from=vacancy_se...
1,Аналитик требований,не указана,Требуемый опыт работы: не требуется; \nПолная ...,✨Присоединяйтесь к Архитекторам Будущего Цифро...,,Аналитическое мышление; \nАналитика; \nСбор и ...,,https://hh.kz/vacancy/88171159?from=vacancy_se...
2,Стажер аналитик-проектировщик (удаленно),от 30000 ₽ на руки,Требуемый опыт работы: не требуется; \nСтажиро...,Привет! Мы занимаемся разработкой и внедрением...,,SQL; \nJavaScript; \n,,https://hh.kz/vacancy/90677195?from=vacancy_se...
3,Системный аналитик,от 30000 до 150000 ₽ до вычета налогов,Требуемый опыт работы: не требуется; \nПолная ...,✨Присоединяйтесь к Архитекторам Будущего Цифро...,,Системный анализ; \nBPMN; \nUML; \nПроектная д...,,https://hh.kz/vacancy/90797243?from=vacancy_se...
4,Бизнес-аналитик,от 30000 до 150000 ₽ до вычета налогов,Требуемый опыт работы: не требуется; \nПолная ...,✨Присоединяйтесь к Архитекторам Будущего Цифро...,,Бизнес-анализ; \nАнализ рынка; \nСоставление б...,,https://hh.kz/vacancy/90797272?from=vacancy_se...
5,Бизнес-аналитик,не указана,Требуемый опыт работы: не требуется; \nПолная ...,"Мы международная команда специалистов, создающ...",,,,https://hh.kz/vacancy/91239733?from=vacancy_se...
6,Финансовый аналитик,от 90000 до 100000 ₽ на руки,Требуемый опыт работы: не требуется; \nПолная ...,Финансовый аналитик (лучшая вакансия для джуна...,,Cash Flow; \nTreasury Management; \nФинансовый...,,https://hh.kz/vacancy/90951167?from=vacancy_se...
7,Аналитик (стратегический консалтинг),не указана,Требуемый опыт работы: не требуется; \nПолная ...,GRP (Global Remote Professionals) - это команд...,,Стратегический консалтинг; \n,,https://hh.kz/vacancy/90616307?from=vacancy_se...
8,Аналитик (стратегический консалтинг),не указана,Требуемый опыт работы: не требуется; \nПолная ...,GRP (Global Remote Professionals) - это команд...,,Стратегический консалтинг; \n,,https://hh.kz/vacancy/90616287?from=vacancy_se...
9,Аналитик (стратегический консалтинг),не указана,Требуемый опыт работы: не требуется; \nПолная ...,GRP (Global Remote Professionals) - это команд...,,Стратегический консалтинг; \n,,https://hh.kz/vacancy/90616138?from=vacancy_se...
